In [17]:

import nltk
import re
%matplotlib inline
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier


from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import FunctionTransformer

In [18]:
df0=pd.read_csv('scraped_data/crawling_0.csv')
df1=pd.read_csv('scraped_data/crawling_1.csv')

'''
df2=pd.read_csv('scraped_data/crawling_2.csv')
df3=pd.read_csv('scraped_data/crawling_3.csv')
df4=pd.read_csv('scraped_data/crawling_4.csv')
df5=pd.read_csv('scraped_data/crawling_5.csv')
df6=pd.read_csv('scraped_data/crawling_6.csv')
df7=pd.read_csv('scraped_data/crawling_7.csv')
df8=pd.read_csv('scraped_data/crawling_8.csv')
df9=pd.read_csv('scraped_data/crawling_9.csv')
df10=pd.read_csv('scraped_data/crawling_10.csv')
df11=pd.read_csv('scraped_data/crawling_11.csv')
data_df=pd.concat([df0,df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11])
'''
data_df=pd.concat([df0,df1])

In [19]:
data_df.reset_index(drop=True, inplace=True)

In [23]:
selected_categories = data_df['class'].value_counts()[:10].index.tolist()
selected_categories

['Thời sự',
 'Thế giới',
 'Pháp luật',
 'Thể thao',
 'Kinh doanh',
 'Sức khỏe',
 'Cần biết',
 'Giáo dục',
 'Văn hóa',
 'Khoa học']

In [24]:
data_df = data_df[data_df['class'].isin(selected_categories)]
data_df

,links,title,description,content,class
0,https://tuoitre.vn/tong-thong-trump-xac-nhan-k...,Tổng thống Trump xác nhận không dự lễ nhậm chứ...,TTO - Tổng thống Mỹ Donald Trump đăng tweet ch...,Sau khi đưa ra cam kết sẽ đảm bảo chuyển giao ...,Thế giới
1,https://tuoitre.vn/dat-nuoc-dat-niem-tin-vao-n...,Đất nước đặt niềm tin vào những học sinh xuất sắc,"TT0 - Tối 8-1, Thủ tướng Chính phủ Nguyễn Xuân...","Chia sẻ tại buổi lễ, Thủ tướng Nguyễn Xuân Phú...",Giáo dục
2,https://tuoitre.vn/luat-su-my-phan-bien-ong-tr...,Luật sư Mỹ phản biện: Ông Trump đâu có kêu ngư...,TTO - Trên chương trình Bill Hemmer Reports củ...,"Ngày 6-1, tình trạng bạo lực đã xảy ra tại tòa...",Thế giới
3,https://tuoitre.vn/thanh-pho-phu-quoc-se-phat-...,Thành phố Phú Quốc sẽ phát triển dựa trên 4 tr...,"TTO - Tối 8-1 tại phường An Thới, chính quyền ...",Phát biểu tại buổi lễ công bố thành lập TP Phú...,Thời sự
4,https://tuoitre.vn/ong-trump-nguoi-ung-ho-toi-...,Ông Trump: Người ủng hộ tôi 'sẽ có tiếng nói t...,TTO - Trong nội dung đăng trên Twitter sau gần...,"""75 triệu người Mỹ yêu nước vĩ đại đã bầu cho ...",Thế giới
...,...,...,...,...,...
11994,/bao-so-10-se-suy-yeu-thanh-ap-thap-khi-di-vao...,Bão số 10 sẽ suy yếu thành áp thấp khi đi vào ...,TTO - Trung tâm Dự báo khí tượng thủy văn quốc...,"Hồi 16h ngày 4-11, vị trí tâm bão ở cách quần ...",Thời sự
11995,/danko-group-trao-quy-hoc-bong-danko-cho-cac-t...,Danko Group trao Quỹ học bổng Danko cho các tr...,Nhằm góp phần xây dựng và phát huy truyền thốn...,Tổng giá trị Quỹ học bổng Danko mà Tập đoàn Da...,Cần biết
11997,/dieu-tra-vu-nam-thanh-nien-bi-trung-13-phat-d...,Điều tra vụ nam thanh niên bị trúng 13 phát đạ...,TTO - Do mâu thuẫn dẫn đến xô xát giữa hai nhó...,"Tối 4-11, Công an tỉnh Quảng Nam cho biết các ...",Pháp luật
11998,/chu-tich-chi-hoi-nghe-ca-lua-ngu-dan-1-6-ti-t...,"Chủ tịch chi hội nghề cá lừa ngư dân 1,6 tỉ ti...",TTO - Chủ tịch Chi hội nghề cá biển thị trấn T...,"Chiều 4-11, Phòng Cảnh sát kinh tế Công an tỉn...",Pháp luật


In [25]:
data_df=data_df[~((data_df['title'].isnull())|
                  (data_df['description'].isnull())|
                  (data_df['content'].isnull()))]

In [26]:
data_df.drop(['links'],axis=1,inplace=True)

C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [27]:
data_df.drop_duplicates(inplace=True)

C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [28]:
data_df

,title,description,content,class
0,Tổng thống Trump xác nhận không dự lễ nhậm chứ...,TTO - Tổng thống Mỹ Donald Trump đăng tweet ch...,Sau khi đưa ra cam kết sẽ đảm bảo chuyển giao ...,Thế giới
1,Đất nước đặt niềm tin vào những học sinh xuất sắc,"TT0 - Tối 8-1, Thủ tướng Chính phủ Nguyễn Xuân...","Chia sẻ tại buổi lễ, Thủ tướng Nguyễn Xuân Phú...",Giáo dục
2,Luật sư Mỹ phản biện: Ông Trump đâu có kêu ngư...,TTO - Trên chương trình Bill Hemmer Reports củ...,"Ngày 6-1, tình trạng bạo lực đã xảy ra tại tòa...",Thế giới
3,Thành phố Phú Quốc sẽ phát triển dựa trên 4 tr...,"TTO - Tối 8-1 tại phường An Thới, chính quyền ...",Phát biểu tại buổi lễ công bố thành lập TP Phú...,Thời sự
4,Ông Trump: Người ủng hộ tôi 'sẽ có tiếng nói t...,TTO - Trong nội dung đăng trên Twitter sau gần...,"""75 triệu người Mỹ yêu nước vĩ đại đã bầu cho ...",Thế giới
...,...,...,...,...
11994,Bão số 10 sẽ suy yếu thành áp thấp khi đi vào ...,TTO - Trung tâm Dự báo khí tượng thủy văn quốc...,"Hồi 16h ngày 4-11, vị trí tâm bão ở cách quần ...",Thời sự
11995,Danko Group trao Quỹ học bổng Danko cho các tr...,Nhằm góp phần xây dựng và phát huy truyền thốn...,Tổng giá trị Quỹ học bổng Danko mà Tập đoàn Da...,Cần biết
11997,Điều tra vụ nam thanh niên bị trúng 13 phát đạ...,TTO - Do mâu thuẫn dẫn đến xô xát giữa hai nhó...,"Tối 4-11, Công an tỉnh Quảng Nam cho biết các ...",Pháp luật
11998,"Chủ tịch chi hội nghề cá lừa ngư dân 1,6 tỉ ti...",TTO - Chủ tịch Chi hội nghề cá biển thị trấn T...,"Chiều 4-11, Phòng Cảnh sát kinh tế Công an tỉn...",Pháp luật


In [29]:
from pyvi import ViTokenizer, ViPosTagger # thư viện NLP tiếng Việt
from tqdm import tqdm
import numpy as np
import gensim # thư viện NLP

In [30]:
data_df["content"][0]

'Sau khi đưa ra cam kết sẽ đảm bảo chuyển giao quyền lực một cách hòa bình, êm thắm, ngày 8-1, ông Trump viết trên Twitter: "Trả lời câu hỏi của các bạn, tôi sẽ không tham dự lễ nhậm chức ngày 20-1".Dòng tweet này của ông liên tục biến động các con số về lượt người xem và chia sẻ cũng như bình luận. Nhiều người phản đối đã để lại bình luận không mấy thân thiện bên dưới như "không ai muốn ngài có mặt ở đó"...Theo Hãng tin Reuters, một nguồn tin am hiểu nội vụ cho biết ở Nhà Trắng đã có sự bàn bạc về việc ông Trump sẽ rời Washington ngày 19-1. Cũng theo nguồn tin này, ông Trump sẽ đến khu nghỉ dưỡng của mình ở bang Florida.Trong lịch sử Mỹ, có nhiều Tổng thống từng quyết định không dự lễ nhậm chức của người kế nhiệm.\xa0Tổng thống Richard Nixon rời Nhà Trắng khi Gerald Ford tuyên thệ. Ông Nixon từ chức, báo với Phó tổng thống của mình là Gerald Ford vào ngày 8-8 và ông Ford thay mình từ "ngày hôm nay".\xa0 Lúc 9h30 ngày 9-8, ông Nixon có bài phát biểu cuối cùng sau đó rời đi. Khi ông đáp

In [31]:
#' '.join(gensim.utils.simple_preprocess(data_df['content'][0]))
def gensim_utils_simple_preprocess(x):
    return ' '.join(gensim.utils.simple_preprocess(x))
data_df['content']=data_df['content'].apply(gensim_utils_simple_preprocess)
data_df['class']=data_df['class'].apply(gensim_utils_simple_preprocess)

C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [32]:
def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word
 
    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word
 
    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            # for index2 in nguyen_am_index:
            #     if index2 != index:
            #         x, y = nguyen_am_to_ids[chars[index]]
            #         chars[index2] = bang_nguyen_am[x][0]
            return ''.join(chars)
 
    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            # chars[nguyen_am_index[1]] = bang_nguyen_am[x][0]
        else:
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
        # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
        chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[2]]]
        # chars[nguyen_am_index[2]] = bang_nguyen_am[x][0]
    return ''.join(chars)

def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True

def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
        Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.
        :param sentence:
        :return:
        """
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\\p{P}*)([p{L}.]*\\p{L}+)(\\p{P}*$)', r'\\1/\\2/\\3', word).split('/')
        # print(cw)
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tu_tieng_viet(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)



data_df['content']=data_df['content'].apply(chuan_hoa_dau_cau_tieng_viet)
data_df['class']=data_df['class'].apply(chuan_hoa_dau_cau_tieng_viet)

C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [33]:
uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"

def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic
dicchar = loaddicchar()
 
# Đưa toàn bộ dữ liệu qua hàm này để chuẩn hóa lại
def covert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

data_df['content']=data_df['content'].apply(covert_unicode)
data_df['class']=data_df['class'].apply(covert_unicode)

C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [34]:
ViTokenizer.tokenize(data_df["content"][0])

'sau khi đưa ra cam_kết sẽ đảm_bảo chuyển_giao quyền_lực một_cách hòa bình êm thắm ngày ông trump viết trên twitter trả_lời câu hỏi của các bạn tôi sẽ không tham_dự lễ nhậm_chức ngày dòng tweet này của ông liên_tục biến_động các con_số về lượt người xem và chia_sẻ cũng như bình_luận nhiều người phản_đối đã để lại bình_luận không mấy thân_thiện bên dưới như không ai muốn ngài có_mặt đó theo hãng tin reuters một nguồn tin am_hiểu nội_vụ cho biết nhà trắng đã có sự bàn_bạc về việc ông trump sẽ rời washington ngày cũng theo nguồn tin này ông trump sẽ đến khu nghỉ_dưỡng của mình bang florida trong lịch_sử mỹ có nhiều tổng_thống từng quyết_định không dự lễ nhậm_chức của người kế_nhiệm tổng_thống richard nixon rời nhà trắng khi gerald ford tuyên_thệ ông nixon từ_chức báo với phó_tổng_thống của mình là gerald ford vào ngày và ông ford thay mình từ ngày hôm_nay lúc ngày ông nixon có bài phát_biểu cuối_cùng sau đó rời đi khi ông đáp máy_bay xuống california ông ford đã tuyên_thệ xong ngoài_ra cò

In [35]:
data_df['content']=data_df['content'].apply(ViTokenizer.tokenize)

C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
data_df['content']

0        sau khi đưa ra cam_kết sẽ đảm_bảo chuyển_giao ...
1        chia_sẻ tại buổi lễ thủ_tướng nguyễn xuân phúc...
2        ngày tình_trạng bạo_lực đã xảy ra tại tòa nhà ...
3        phát_biểu tại buổi lễ công_bố thành_lập tp phú...
4        triệu người mỹ yêu nước vĩ_đại đã bầu cho tôi ...
                               ...                        
11994    hồi ngày vị_trí tâm_bão cách quần_đảo hoàng_sa...
11995    tổng giá_trị quỹ học_bổng danko mà tập_đoàn da...
11997    tối công_an tỉnh quảng_nam cho biết các đơn_vị...
11998    chiều phòng cảnh_sát kinh_tế công_an tỉnh thừa...
11999    trao_đổi với tuổi_trẻ online ông khổng minh tu...
Name: content, Length: 6550, dtype: object

In [39]:
y_sr = data_df["class"] # sr là viết tắt của series
X_df = data_df["content"]
train_X_df, val_X_df, train_y_sr, val_y_sr = train_test_split(X_df, y_sr, test_size=0.3,stratify=y_sr, random_state=0)

In [40]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(train_X_df)

# transform the training and validation data using count vectorizer object
X_data_count = count_vect.transform(train_X_df)
X_test_count = count_vect.transform(val_X_df)

In [41]:
# word level - we choose max number of words equal to 30000 except all words (100k+ words)
tfidf_vect = TfidfVectorizer(analyzer='word', max_features=30000)
tfidf_vect.fit(train_X_df) # learn vocabulary and idf from training set
X_data_tfidf =  tfidf_vect.transform(train_X_df)
# assume that we don't have test set before
X_test_tfidf =  tfidf_vect.transform(val_X_df)

In [43]:
# ngram level - we choose max number of words equal to 30000 except all words (100k+ words)
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', max_features=30000, ngram_range=(2, 3))
tfidf_vect_ngram.fit(train_X_df)
X_data_tfidf_ngram =  tfidf_vect_ngram.transform(train_X_df)
# assume that we don't have test set before
X_test_tfidf_ngram =  tfidf_vect_ngram.transform(val_X_df)

In [44]:
# ngram-char level - we choose max number of words equal to 30000 except all words (100k+ words)
tfidf_vect_ngram_char = TfidfVectorizer(analyzer='char', max_features=30000, ngram_range=(2, 3))
tfidf_vect_ngram_char.fit(train_X_df)
X_data_tfidf_ngram_char =  tfidf_vect_ngram_char.transform(train_X_df)
# assume that we don't have test set before
X_test_tfidf_ngram_char =  tfidf_vect_ngram_char.transform(val_X_df)

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=300, random_state=42)
svd.fit(X_data_tfidf)


X_data_tfidf_svd = svd.transform(X_data_tfidf)
X_test_tfidf_svd = svd.transform(X_test_tfidf)